This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
# api key removed prior to saving and uploading to github
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
# import requests for url requests
import requests

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

# set limit=1 to only return one row/day
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?limit=1&api_key=' + API_KEY
r = requests.get(url)
json_data = r.json()
print(json_data)

{'dataset': {'id': 10095370, 'dataset_code': 'AFX_X', 'database_code': 'FSE', 'name': 'Carl Zeiss Meditec (AFX_X)', 'description': 'Stock Prices for Carl Zeiss Meditec (2019-09-25) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704', 'refreshed_at': '2019-10-09T22:19:01.336Z', 'newest_available_date': '2019-10-09', 'oldest_available_date': '2000-06-07', 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'frequency': 'daily', 'type': 'Time Series', 'premium': False, 'limit': 1, 'transform': None, 'column_index': None, 'start_date': '2000-06-07', 'end_date': '2019-10-09', 'data': [['2019-10-09', 106.3, 107.7, 105.5, 107.0, None, 94098.0, 10050758.0, None, None, None]], 'collapse': None, 'order': None, 'database_id': 6129}}


In [ ]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

# everything is inside the 'dataset' index
# the column names are provided as a list in the 'column_names' index inside 'dataset'
# the data is provided as separate lists for each day in the 'data' index inside 'dataset'

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [4]:
# Task 1: Collect the data for 2017

# set the url with an easy to modify structure
# set url_end to only retrive one day's worth of data by passing a variable/value
url_start = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json'
url_end = '?start_date=2017-01-01&end_date=2017-12-31&api_key=' + API_KEY
url2 = url_start + url_end

# create the request object
r2 = requests.get(url2)

# get the json data
json_data = r2.json()

# display the start and end date to verify the url only requested one day's worth of data
# use the first date index and the last date index for verification
print('End Date: ', json_data['dataset']['data'][0][0])
print('Start Date: ', json_data['dataset']['data'][-1][0])

End Date:  2017-12-29
Start Date:  2017-01-02


In [5]:
# Task 2: Convert the data into a dictionary
# column names are the keys
# data are in lists nested below each key

# store column names in a list to use as keys for the dictionary
keys = json_data['dataset']['column_names']

# store the data in a variable for easier processing
data = json_data['dataset']['data']

# extract the data each column becomes a list
extracted_data = [[data[i][y] for i in range(len(data))] for y in range(len(keys))]

# nest each column as the value of the associated column name (dictionary key)
afx = {keys[y]: extracted_data[y] for y in range(len(keys))}

In [6]:
# Task 3: Calculate the highest and lowest opening prices during this period

# store the opening prices in a list
op = afx['Open']

# remove all of the 'None' values
op_rmv = [i for i in op if i != None]

# calculate the max and min values
max_open = max(op_rmv)
min_open = min(op_rmv)

# format and print the answer
print('Max Opening Price for 2017: ', '$' + "{:6.2f}".format(max_open))
print('Min Opening Price for 2017: ', '$' + "{:6.2f}".format(min_open))

Max Opening Price for 2017:  $ 53.11
Min Opening Price for 2017:  $ 34.00


In [7]:
# Task 4: Calculate the largest change during one day
# based on 'High' and 'Low' columns

# calculate the change for each day and populate a list
# selecting directly from the dictionary rather than creating new lists first
change = [afx['High'][i] - afx['Low'][i] for i in range(len(afx['High']))]    

# see if the greatest change was positive or negative for reporting
pos_change = max(change)
neg_change = min(change)

# set the result of the greatest change
if abs(pos_change) > abs(neg_change):
    result = '+' + str(max(change))
else:
    result = min(change)

# print the result
print('The largest change in one day was: ', result)

The largest change in one day was:  +2.8100000000000023


In [8]:
# Task 5: Calculate the largest change between any two consecutive closing prices
# use the 'Close' column

# assign the list in question to a variable to use in the list comprehension
close = afx['Close']

# generate a list of the changes between every set of days
changes = [close[i]-close[i+1] for i in range(len(close)-1)]

# check if the greatest change was positive or negative for reporting
pos_change2 = max(changes)
neg_change2 = min(changes)

# set the result for the greatest change
if abs(pos_change2) > abs(neg_change2):
    result2 = '+' + str(max(changes))
else:
    result2 = min(changes)

# print the result
print('The largest change in closing price between any two days in 2017 was: ', result2)

The largest change in closing price between any two days in 2017 was:  -2.559999999999995


In [9]:
# Task 6: Calculate the average daily trading volume during 2017
# use 'Traded Volume' column
# could have used the mean() function from the statistics module
# avoiding module useage in accordance with project guidelines

vol = afx['Traded Volume']
print('The average daily trading volume for 2017 was: ', round(sum(vol)/len(vol), 1))

The average daily trading volume for 2017 was:  89124.3


In [10]:
# Task 7: Calculate the median trading volume for 2017
# need index=127 on the ranked data

# store the index value for the median when the data are ranked
idx = len(vol)//2

# sort the list of volumes
vol_sort = sorted(vol)

# access the median value
med_vol = vol_sort[idx]

# print the result
print('The median daily trading volume for 2017 was: ', med_vol)

The median daily trading volume for 2017 was:  76286.0
